In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout, LSTM, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical

from cv2 import imread, resize

In [2]:
data = pd.read_csv('../input/water-quality/waterQuality1.csv')

data = data.drop('ammonia', axis = 1)
data = data.drop('flouride', axis = 1)
data = data.drop('selenium', axis = 1)
data = data.drop('uranium', axis = 1)

data = data[data['is_safe'] != '#NUM!']
data['is_safe'] = data['is_safe'].astype(int)

columns = dict(zip(list(range(data.shape[1])), data.columns.values.tolist()))
is_safe = data['is_safe']

data = pd.DataFrame(StandardScaler().fit_transform(data)).rename(columns = columns)
data['is_safe'] = is_safe
data = data.dropna()
data.head(6)


In [3]:
X = data.drop('is_safe', axis = 1)
Y = data['is_safe']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 13)

In [4]:
def build_():
    model = Sequential()
    
    model.add(Dense(256, input_dim = 16))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(2, activation = 'sigmoid'))
    
    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'sgd',
        metrics = [keras.metrics.Recall(name = 'recall')]
    )
    return model


class MLP():
    def __init__(self, epochs, batch_size):
        self.clf = KerasClassifier(build_, epochs = epochs, batch_size = batch_size)
        
    def fit_(self, x, y):
        self.clf.fit(x, y, verbose = 0)
    
    def predict_(self, x, y):
        self.p = self.clf.predict(x)
        return np.sum(y == self.p) / len(y)
        
        
clf = MLP(epochs = 250, batch_size = 32)
clf.fit_(X_train, Y_train)

clf.predict_(X_test, Y_test)


In [5]:
path = '../input/flowers-recognition/flowers'
classes = np.array(os.listdir(path))
for i, j in enumerate(classes):
    print(i, j)

In [6]:
X = []
Y = []

for i, j in enumerate(classes):
    folder = path + '/' + str(j)
    for image in os.listdir(folder):
        img = imread((os.path.join(folder, image)), 1)[...,::-1] / 255.0
        img = resize(img, (100, 100))
        X.append(img)
        Y.append(i)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 13)

X_train = np.array(X_train)
Y_train = to_categorical(np.array(Y_train))
X_test = np.array(X_test)

In [8]:
def build_():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape = (100, 100, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation = 'softmax'))

    model.compile(
            loss = 'categorical_crossentropy',
            optimizer = 'rmsprop',
            metrics = [keras.metrics.Recall(name = 'recall'), 
                       keras.metrics.Accuracy(name = 'accuracy')]
        )

    return model


class CNN():
    def __init__(self, epochs, batch_size):
        self.clf = KerasClassifier(build_, epochs = epochs, batch_size = batch_size)
        
    def fit_(self, x, y):
        self.clf.fit(x, y, verbose = 0)
    
    def predict_(self, x, y):
        self.p = self.clf.predict(x)
        return np.sum(y == self.p) / len(y)
        
        
clf = CNN(epochs = 25, batch_size = 16)
clf.fit_(X_train, Y_train)

clf.predict_(X_test, Y_test)

In [10]:
df = pd.read_csv('../input/time-series-forecasting-with-yahoo-stock-price/yahoo_stock.csv')
df = df.drop(['Volume', 'Adj Close'], axis = 1)
df.Date = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

df

In [11]:
df.isnull().sum()

In [12]:
df[['High','Low','Open','Close']].plot(figsize = (16, 6))

In [13]:
sc = MinMaxScaler(feature_range=(0,1))

def load_data(datasetname, column, seq_len):
    # A support function to help prepare datasets for an RNN/LSTM/GRU
    data = datasetname.loc[:,column]

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    result = np.array(result)

    #Last 10% is used for validation test, first 90% for training
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    return [x_train, y_train, x_test, y_test]

In [14]:
X_train, Y_train, X_test, Y_test = load_data(df, 'Close', 50)

df['Close'][:int(len(df['Close']) * .9)].plot(figsize=(16,4),legend=True)
df['Close'][int(len(df['Close']) * .9):].plot(figsize=(16,4),legend=True)
plt.legend(['Training set (First 90%)','Test set (Last 10%)'])
plt.show()

In [15]:
model=Sequential()

model.add(LSTM(25, activation = 'relu', input_shape = (X_train.shape[1],1)))
model.add(Dense(10))
model.add(Dense(1))

model.compile(
    loss = 'mean_squared_error',
    optimizer = 'adam'
)


model.fit(X_train, Y_train, epochs=10, batch_size=32)


In [16]:
prediction = model.predict(X_test)


plt.plot(Y_test, label='Test Data') 
plt.plot(prediction, label='Prediction') 
plt.legend()
plt.show() 